In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('RawData/data.csv')

In [3]:
# columns that cannot be used for price prediction
columns_to_drop = ['displayColor', 'city', 'lat', 'lon', 'regionName', 'experience', 'priceMobile', 'recentPriceDrop',
                   'eligibleForFinancing', 'financingExperience', 'isHot', 'newPriceAsMsrp', 'active', 'state', 'trim', 
                   'monthlyPayment', 'bodyType', 'mileage', 'mileageHumanized', 'showNewMileage', 'id', 'price']

In [4]:
data.drop(columns_to_drop, inplace=True, axis=1)

In [5]:
data

,vin,year,make,model,condition,modelId,bodyStyle,mileageUnformatted,priceUnformatted,createdAt,updatedAt
0,W1N4M4GB4MW095064,2021,Mercedes-Benz,GLB,used,2356.0,suv,51482.0,27499,2023-10-19T17:50:59.833Z,2023-12-16T18:55:28.092Z
1,1N4AL3AP3FC220920,2015,Nissan,Altima,used,1511.0,sedan,126966.0,6489,2023-09-20T15:10:50.471Z,2023-12-18T16:31:38.609Z
2,WA1LAAF79HD055629,2017,Audi,Q7,used,1371.0,suv,72937.0,22498,2023-12-16T16:29:43.210Z,2023-12-19T02:24:43.301Z
3,WDC0G4JB4KV142709,2019,Mercedes-Benz,GLC,used,2104.0,suv,32284.0,25998,2023-12-16T02:30:04.978Z,2023-12-16T02:30:04.978Z
4,WP0AD2A76FL040137,2015,Porsche,Panamera,used,1882.0,sedan,86099.0,24999,2023-07-25T18:32:06.185Z,2023-12-14T02:27:35.963Z
...,...,...,...,...,...,...,...,...,...,...,...
35252,1FTFW1E86PFD11758,2023,Ford,F-150,new,957.0,truck,5.0,74082,2023-10-20T17:47:06.626Z,2024-01-14T18:57:40.311Z
35253,1FTFW1EDXPFD06625,2023,Ford,F-150,new,957.0,truck,5.0,61617,2023-10-20T17:47:06.646Z,2024-01-14T18:57:40.457Z
35254,1FTVW1EVXPWG36558,2023,Ford,F-150 Lightning,new,2441.0,truck,45.0,79290,2023-09-20T17:23:39.333Z,2024-01-18T18:58:15.820Z
35255,2FMPK4G91RBA17174,2024,Ford,Edge,new,955.0,suv,6.0,37940,2023-08-19T17:19:30.458Z,2024-01-19T18:58:16.884Z


In [6]:
data.columns

Index(['vin', 'year', 'make', 'model', 'condition', 'modelId', 'bodyStyle',
       'mileageUnformatted', 'priceUnformatted', 'createdAt', 'updatedAt'],
      dtype='object')

In [7]:
dummy=data.groupby(['vin']).agg({'updatedAt': ['max']}).reset_index()
dummy.columns=dummy.columns.droplevel(1)

In [8]:
result=data.merge(dummy, how='inner', on=['vin', 'updatedAt']).drop_duplicates().drop(['createdAt', 'updatedAt'], axis=1)

In [9]:
result

,vin,year,make,model,condition,modelId,bodyStyle,mileageUnformatted,priceUnformatted
0,W1N4M4GB4MW095064,2021,Mercedes-Benz,GLB,used,2356.0,suv,51482.0,27499
1,19UUB3F57FA003319,2015,Acura,TLX,used,2060.0,sedan,73390.0,16499
2,5UXWX9C52F0D57109,2015,BMW,X3,used,1323.0,suv,81653.0,14499
3,JTHBA1D28H5060662,2017,Lexus,IS 200t,used,2109.0,sedan,41763.0,22399
4,WBXYH9C09L5P32701,2020,BMW,X2,used,2201.0,suv,55228.0,21995
...,...,...,...,...,...,...,...,...,...
12648,1FTFW1E86PFD11758,2023,Ford,F-150,new,957.0,truck,5.0,74082
12649,1FTFW1EDXPFD06625,2023,Ford,F-150,new,957.0,truck,5.0,61617
12650,1FTVW1EVXPWG36558,2023,Ford,F-150 Lightning,new,2441.0,truck,45.0,79290
12651,2FMPK4G91RBA17174,2024,Ford,Edge,new,955.0,suv,6.0,37940


In [10]:
result.to_csv('CleanData/basic.csv', index=False)

In [11]:
result['make'].nunique()

41

In [12]:
result['model'].nunique()

489

## How many VIN numbers from features data is in price data?

In [13]:
ft_data=pd.read_csv('RawData/features_data.csv')

In [14]:
vin=result['vin'].values
vin_ft=ft_data['vin'].values

In [15]:
dummy=[v for v in vin_ft if v in vin]

In [16]:
len(dummy)

10191